In [26]:
%%time
#global variables
startx=0
starty=0
rowsize=128
colsize=128
xoverlap=12
yoverlap=16
nodataval=0
bandnumber=1
file="JT1_2020-04-23"
fp = r'JT1_2020-04-23.tif'

# essential libraries
import rasterio
from rasterio.plot import show
import numpy as np
import math
import shapely
from shapely.geometry import box
from rasterio.mask import mask
import json
import os

# to disable shapely error createlinear Ring returning a null pointer
shapely.speedups.disable()

# initial required variable
img = rasterio.open(fp)
rows=img.shape[0]
cols=img.shape[1]
bands=img.count
index=1
blockslist=[]

# iterates over all the boxes
while(startx<rows-xoverlap and starty<cols-yoverlap):
#   handling corner blocks
    if rows-startx < rowsize:
        startx=rows-rowsize
    if cols-starty < colsize:
        starty=cols-colsize

#   mask out window with given initial coordinates and count nonzero numbers in numpy array
    window=rasterio.windows.Window(starty,startx,colsize,rowsize)
    bounds=img.window_bounds(window)
    geom=box(*bounds)
    windowdata = img.read(window=window)
    count=np.count_nonzero(windowdata[0])
    
#   process required statistics
    idx=math.floor(startx/(rowsize-xoverlap))
    idy=math.floor(starty/(colsize-yoverlap))
    no_data_perc = 100 - (100*count)/(rowsize*colsize)
    dic={
        "index":index,
        "startx":startx,
        "starty":starty,
        "idx":idx,
        "idy":idy,
        "no-data-percentage":no_data_perc,
    }
    blockslist.append(dic)
    
#   update for next blockset
    starty+=colsize-yoverlap  
    index+=1
    if cols-yoverlap==starty:
        starty=0
        startx+=rowsize-xoverlap

# defining data for json
json_data = {
    "filename": file,
    "filelocation":os.path.realpath('.'),
    "width":colsize,
    "height":rowsize,
    "xoverlap": xoverlap,
    "yoverlap": yoverlap,
}
ind=1
# adding each block's data
for i in blockslist:
    x={
        "tile"+str(ind):i,
    }
    ind+=1
    json_data.update(x)

# write to json file
with open('data.json', 'w') as jsonFile:
    json.dump(json_data, jsonFile,indent=4)
    jsonFile.close()


Wall time: 272 ms
